# Defaulting

In [1]:
import anndata
import matplotlib.pyplot as plt
import numpy as np
import scanpy as sc
from scipy.stats import spearmanr
from scvi.data import cortex, smfish
from scvi.external import GIMVI

/home/jaecheon/anaconda3/envs/scenvi/lib/python3.10/site-packages/docrep/decorators.py:43: SyntaxWarning: 'param_categorical_covariate_keys' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)
/home/jaecheon/anaconda3/envs/scenvi/lib/python3.10/site-packages/docrep/decorators.py:43: SyntaxWarning: 'param_continuous_covariate_keys' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)


## loading

In [4]:
st_data=sc.read_h5ad("/data/kjc2/projects/P330.CSA/rds/250210_CRC_BJM_0050585_Region1_25-04-14-16-53.h5ad")
# sc_data=sc.read_h5ad("/data/kjc2/projects/P330.CSA/rds/downsampled_100_25-04-15-08-34.h5ad")
sc_data=sc.read_h5ad("/data/kjc2/projects/P330.CSA/rds/downsampled_100_re_processed_log1p_harmonized.h5ad")
print(sc_data)
print(st_data)

AnnData object with n_obs × n_vars = 42649 × 28476
    obs: 'dataset', 'medical_condition', 'cancer_type', 'sample_id', 'sample_type', 'tumor_source', 'replicate', 'sample_tissue', 'anatomic_region', 'anatomic_location', 'tumor_stage', 'tumor_stage_TNM', 'tumor_stage_TNM_T', 'tumor_stage_TNM_N', 'tumor_stage_TNM_M', 'tumor_size', 'tumor_dimensions', 'tumor_grade', 'histological_type', 'microsatellite_status', 'mismatch_repair_deficiency_status', 'MLH1_promoter_methylation_status', 'MLH1_status', 'KRAS_status', 'BRAF_status', 'APC_status', 'TP53_status', 'PIK3CA_status', 'SMAD4_status', 'NRAS_status', 'MSH6_status', 'FBXW7_status', 'NOTCH1_status', 'MSH2_status', 'PMS2_status', 'POLE_status', 'ERBB2_status', 'STK11_status', 'HER2_status', 'CTNNB1_status', 'BRAS_status', 'patient_id', 'sex', 'age', 'treatment_status_before_resection', 'treatment_drug', 'treatment_response', 'RECIST', 'platform', 'platform_fine', 'cellranger_version', 'reference_genome', 'matrix_type', 'enrichment_cell_ty

In [12]:
print(sc_data.obs.index)
print(sc_data.obs_names)

Index(['VUMC_HTAN_cohort3-HTA11_5679_2000001011-ATCCGCTAGTCTCTCT-7133-YX-24-7133',
       'Chen_2024-GSM7558328-ACACCAATCATTTGGG',
       'WUSTL_HTAN-HTA12_106_2-GTGTGATGTGATGGCA',
       'Qin_2023-XFZ_26_PRET_lib_1_web_0_CELL1300_N1',
       'Zhang_2018_CD3Pos-GSM3356226-TTY63-20160909',
       'Elmentaite_2021_10x5p_CD45Neg-ERS12382602-CAACCAATCAGGCCCA',
       'Wang_2023_CD45Pos-s0920_SampleTag10-111003s7-5_7',
       'Wang_2023_CD45Pos-s1231_SampleTag10-417137s3-3_4',
       'Wang_2023_CD45Pos-s0115_SampleTag04-415515s15-3_4',
       'Chen_2024-GSM7558393-GTAACGTCAAGCTGAG',
       ...
       'Wang_2023_CD45Pos-s0115_SampleTag04-173682s15-3_4',
       'Joanito_2022_SG1-MUX9065-ATGCGATAGGTGCTTT', 'MUI_Innsbruck-P8-898602',
       'Chen_2024-GSM7558476-ATCTGCCGTTCGGGCT',
       'VUMC_HTAN_CRC-HTA11_99999971662_82457-TGATGTGGCTAGTGGATGG',
       'Chen_2024-GSM7558476-TGTGGTAGTCATATGC',
       'Borras_2023_SMC5-SMC174T-ACATACGGTCATCGGC',
       'Qin_2023-XFZ_41_POSTT_lib_3_web_1_CELL411

In [5]:
train_size = 0.8

%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}
%config InlineBackend.figure_format='retina'

# Preparing data
In this section, we hold out some of the genes in the spatial dataset in order to test the imputation results

## metadata handling

In [21]:
seq_gene_names = seq_data.var_names
n_genes = seq_data.n_vars
n_train_genes = int(n_genes * train_size)


In [38]:
spatial_data_partial

AnnData object with n_obs × n_vars = 25620 × 771
    obs: 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'genomic_control_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'nucleus_count', 'segmentation_method', 'n_genes_by_counts', 'n_genes', 'n_counts', 'leiden_res0.8', 'batch', '_scvi_batch'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'mean', 'std'
    uns: 'hvg', 'leiden_res0.8', 'leiden_res0.8_colors', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap', 'spatial'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [33]:
spatial_data_partial.obs["batch"]="Xenium"

In [ ]:
spatial_data_partial.obs["batch"]="Xenium"

In [37]:
seq_data

AnnData object with n_obs × n_vars = 18035 × 964
    obs: 'dataset', 'medical_condition', 'cancer_type', 'sample_id', 'sample_type', 'tumor_source', 'replicate', 'sample_tissue', 'anatomic_region', 'anatomic_location', 'tumor_stage', 'tumor_stage_TNM', 'tumor_stage_TNM_T', 'tumor_stage_TNM_N', 'tumor_stage_TNM_M', 'tumor_size', 'tumor_dimensions', 'tumor_grade', 'histological_type', 'microsatellite_status', 'mismatch_repair_deficiency_status', 'MLH1_promoter_methylation_status', 'MLH1_status', 'KRAS_status', 'BRAF_status', 'APC_status', 'TP53_status', 'PIK3CA_status', 'SMAD4_status', 'NRAS_status', 'MSH6_status', 'FBXW7_status', 'NOTCH1_status', 'MSH2_status', 'PMS2_status', 'POLE_status', 'ERBB2_status', 'STK11_status', 'HER2_status', 'CTNNB1_status', 'BRAS_status', 'patient_id', 'sex', 'age', 'treatment_status_before_resection', 'treatment_drug', 'treatment_response', 'RECIST', 'platform', 'platform_fine', 'cellranger_version', 'reference_genome', 'matrix_type', 'enrichment_cell_type

In [ ]:
seq_data.obs["leiden"]=seq_data.obs["leiden"]

In [32]:
seq_data.obs["batch"]=seq_data.obs["dataset"]

In [31]:
sorted(seq_data.obs.columns.tolist())

['APC_status',
 'BRAF_status',
 'BRAS_status',
 'CMS_type',
 'CTNNB1_status',
 'ENA_sample_accession',
 'ERBB2_status',
 'FBXW7_status',
 'G2M_score',
 'GEO_sample_accession',
 'HER2_status',
 'KRAS_status',
 'MLH1_promoter_methylation_status',
 'MLH1_status',
 'MSH2_status',
 'MSH6_status',
 'NCBI_BioProject_accession',
 'NOTCH1_status',
 'NRAS_status',
 'PIK3CA_status',
 'PMS2_status',
 'POLE_status',
 'RECIST',
 'SMAD4_status',
 'SOLO_doublet_prob',
 'SOLO_doublet_status',
 'SOLO_singlet_prob',
 'SRA_sample_accession',
 'STK11_status',
 'S_score',
 'TP53_status',
 'age',
 'anatomic_location',
 'anatomic_region',
 'assay',
 'assay_ontology_term_id',
 'cancer_type',
 'cell_type',
 'cell_type_coarse',
 'cell_type_coarse_study',
 'cell_type_fine',
 'cell_type_middle',
 'cell_type_middle_study',
 'cell_type_ontology_term_id',
 'cell_type_predicted',
 'cell_type_study',
 'cellranger_version',
 'country',
 'dataset',
 'development_stage',
 'development_stage_ontology_term_id',
 'disease',


In [17]:
# only use genes in both datasets
common_genes = sc_data.var_names.intersection(st_data.var_names)
seq_data = sc_data[:, common_genes].copy()

seq_gene_names = seq_data.var_names
n_genes = seq_data.n_vars
n_train_genes = int(n_genes * train_size)

# randomly select training_genes
rand_train_gene_idx = np.random.choice(range(n_genes), n_train_genes, replace=False)
rand_test_gene_idx = sorted(set(range(n_genes)) - set(rand_train_gene_idx))
rand_train_genes = seq_gene_names[rand_train_gene_idx]
rand_test_genes = seq_gene_names[rand_test_gene_idx]

# spatial_data_partial has a subset of the genes to train on
spatial_data_partial = st_data[:, rand_train_genes].copy()

# remove cells with no counts
scanpy.pp.filter_cells(spatial_data_partial, min_counts=1)
scanpy.pp.filter_cells(seq_data, min_counts=1)

# setup_anndata for spatial and sequencing data
GIMVI.setup_anndata(spatial_data_partial, labels_key="labels", batch_key="batch")
GIMVI.setup_anndata(seq_data, labels_key="labels")

# spatial_data should use the same cells as our training data
# cells may have been removed by scanpy.pp.filter_cells()
st_data = st_data[spatial_data_partial.obs_names]

KeyError: "Values ['A2ML1', 'AAMP', 'AAR2', 'AARSD1', 'ABAT', 'ABCA3', 'ABCA4', 'ABCB6', 'ABCC12', 'ABCC3', 'ABCC4', 'ABCC6', 'ABCC8', 'ABCC9', 'ABCD1', 'ABCD3', 'ABCD4', 'ABCF3', 'ABCG2', 'ABHD11', 'ABHD6', 'ABI3', 'ABL1', 'ABO', 'ABTB2', 'ACAA2', 'ACACA', 'ACACB', 'ACAP2', 'ACAT1', 'ACE', 'ACE2', 'ACHE', 'ACKR3', 'ACLY', 'ACOD1', 'ACPP', 'ACRBP', 'ACRV1', 'ACSL4', 'ACTBL2', 'ACTL6A', 'ACTL7A', 'ACTN1', 'ACTN2', 'ACTN4', 'ACVR1', 'ACVR1B', 'ACVR2A', 'ACVRL1', 'ACYP2', 'ADA', 'ADAM10', 'ADAM12', 'ADAM17', 'ADAM9', 'ADAMTS12', 'ADAMTS19', 'ADAMTS2', 'ADAMTS4', 'ADAMTS5', 'ADAMTSL3', 'ADAR', 'ADCY2', 'ADCYAP1R1', 'ADD1', 'ADGRA1', 'ADGRA2', 'ADGRA3', 'ADGRB1', 'ADGRB2', 'ADGRD1', 'ADGRD2', 'ADGRE1', 'ADGRG5', 'ADGRG6', 'ADGRL1', 'ADGRL4', 'ADIPOQ', 'ADIPOR1', 'ADIPOR2', 'ADNP', 'ADORA1', 'ADORA2A', 'ADORA2B', 'ADORA3', 'ADRA1A', 'ADRA1B', 'ADRA1D', 'ADRA2A', 'ADRA2B', 'ADRA2C', 'ADRB1', 'ADRB3', 'ADTRP', 'AFAP1L2', 'AFG1L', 'AFM', 'AGAP1', 'AGAP3', 'AGER', 'AGMAT', 'AGO1', 'AGRP', 'AGTR1

In [39]:
# setup_anndata for spatial and sequencing data
GIMVI.setup_anndata(spatial_data_partial, labels_key="leiden_res0.8", batch_key="batch")
GIMVI.setup_anndata(seq_data, labels_key="leiden")

# spatial_data should use the same cells as our training data
# cells may have been removed by scanpy.pp.filter_cells()
st_data = st_data[spatial_data_partial.obs_names]

/home/jaecheon/anaconda3/envs/scenvi/lib/python3.10/site-packages/scvi/data/fields/_base_field.py:63: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)
/home/jaecheon/anaconda3/envs/scenvi/lib/python3.10/site-packages/scvi/data/fields/_base_field.py:63: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)
